In [1]:
import os
import time
import argparse, parser
from options.train_dicts import TrainOptions
from data import create_dataset
from easydict import EasyDict
from models import create_model
from util.visualizer import Visualizer

In [2]:
params = EasyDict()
params.dataroot = '/media/data2/dataset/maps'
params.name = 'maps_pix2pix'
params.model = 'pix2pix'
params.direction = 'BtoA'
params.gpu_ids = '3'

In [3]:
opt = TrainOptions().parse(params)

----------------- Options ---------------
               batch_size: 1                             
                    beta1: 0.5                           
          checkpoints_dir: ./checkpoints                 
           continue_train: False                         
                crop_size: 256                           
                 dataroot: /media/data2/dataset/maps     
             dataset_mode: aligned                       
                direction: BtoA                          
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4                             
             display_port: 8888                          
           display_server: http://localhost              
          display_winsize: 256                           
                    epoch: latest                        
              epoch_count: 1  

In [4]:
dataset = create_dataset(opt)
dataset_size = len(dataset)
print('The number of training images = %d' % dataset_size)

dataset [AlignedDataset] was created
The number of training images = 2192


In [5]:
model = create_model(opt)

initialize network with normal
initialize network with normal
model [Pix2PixModel] was created


In [6]:
model.setup(opt)

---------- Networks initialized -------------
DataParallel(
  (module): UnetGenerator(
    (model): UnetSkipConnectionBlock(
      (model): Sequential(
        (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
        (1): UnetSkipConnectionBlock(
          (model): Sequential(
            (0): LeakyReLU(negative_slope=0.2, inplace=True)
            (1): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
            (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (3): UnetSkipConnectionBlock(
              (model): Sequential(
                (0): LeakyReLU(negative_slope=0.2, inplace=True)
                (1): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
                (2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (3): UnetSkipConnectionBlock(
                  (model): Sequential(
    

In [7]:
total_iters = 0

In [ ]:
for epoch in range(opt.epoch_count, opt.n_epochs + opt.n_epochs_decay + 1):    # outer loop for different epochs; we save the model by <epoch_count>, <epoch_count>+<save_latest_freq>
        epoch_start_time = time.time()  # timer for entire epoch
        iter_data_time = time.time()    # timer for data loading per iteration
        epoch_iter = 0                  # the number of training iterations in current epoch, reset to 0 every epoch

        for i, data in enumerate(dataset):  # inner loop within one epoch
            iter_start_time = time.time()  # timer for computation per iteration
            if total_iters % opt.print_freq == 0:
                t_data = iter_start_time - iter_data_time

            total_iters += opt.batch_size
            epoch_iter += opt.batch_size
            model.set_input(data)         # unpack data from dataset and apply preprocessing
            model.optimize_parameters()   # calculate loss functions, get gradients, update network weights

            if total_iters % opt.display_freq == 0:   # display images on visdom and save images to a HTML file
                save_result = total_iters % opt.update_html_freq == 0
                model.compute_visuals()

            if total_iters % opt.print_freq == 0:    # print training losses and save logging information to the disk
                losses = model.get_current_losses()
                t_comp = (time.time() - iter_start_time) / opt.batch_size

            if total_iters % opt.save_latest_freq == 0:   # cache our latest model every <save_latest_freq> iterations
                print('saving the latest model (epoch %d, total_iters %d)' % (epoch, total_iters))
                save_suffix = 'iter_%d' % total_iters if opt.save_by_iter else 'latest'
                model.save_networks(save_suffix)

            iter_data_time = time.time()
        if epoch % opt.save_epoch_freq == 0:              # cache our model every <save_epoch_freq> epochs
            print('saving the model at the end of epoch %d, iters %d' % (epoch, total_iters))
            model.save_networks('latest')
            model.save_networks(epoch)

        print('End of epoch %d / %d \t Time Taken: %d sec' % (epoch, opt.n_epochs + opt.n_epochs_decay, time.time() - epoch_start_time))
        model.update_learning_rate()                     # update learning rates at the end of every epoch.

End of epoch 1 / 200 	 Time Taken: 85 sec
learning rate = 0.0002000
End of epoch 2 / 200 	 Time Taken: 83 sec
learning rate = 0.0002000
saving the latest model (epoch 3, total_iters 5000)
End of epoch 3 / 200 	 Time Taken: 84 sec
learning rate = 0.0002000
End of epoch 4 / 200 	 Time Taken: 80 sec
learning rate = 0.0002000
saving the latest model (epoch 5, total_iters 10000)
saving the model at the end of epoch 5, iters 10960
End of epoch 5 / 200 	 Time Taken: 88 sec
learning rate = 0.0002000
End of epoch 6 / 200 	 Time Taken: 83 sec
learning rate = 0.0002000
saving the latest model (epoch 7, total_iters 15000)
End of epoch 7 / 200 	 Time Taken: 83 sec
learning rate = 0.0002000
End of epoch 8 / 200 	 Time Taken: 82 sec
learning rate = 0.0002000
End of epoch 9 / 200 	 Time Taken: 84 sec
learning rate = 0.0002000
saving the latest model (epoch 10, total_iters 20000)
saving the model at the end of epoch 10, iters 21920
End of epoch 10 / 200 	 Time Taken: 85 sec
learning rate = 0.0002000
En